In [2]:
import pygame
import random
import math

pygame.init()

# --- ตั้งค่าหน้าจอ ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Text or Die - Tower Camera Edition")

# --- สี ---
WHITE = (255, 255, 255)
BLACK = (10, 12, 24)
BLUE = (30, 70, 200, 180)
LIGHT_BLUE = (100, 180, 255)
GREY = (180, 180, 180)
RED = (255, 80, 80)
GREEN = (80, 255, 120)
YELLOW = (255, 255, 120)

# --- ฟอนต์ ---
FONT = pygame.font.SysFont("arial", 32)
BIG = pygame.font.SysFont("arial", 64)

# --- นาฬิกา ---
clock = pygame.time.Clock()

# --- คำถามและคำตอบ ---
questions = {
    "Name a fruit": ["apple", "banana", "orange", "mango", "grape", "kiwi"],
    "Name a country": ["thailand", "japan", "korea", "china", "france", "italy"],
    "Name an animal": ["dog", "cat", "elephant", "tiger", "lion", "rabbit"]
}

# --- ตัวแปรบล็อก ---
block_width = 50
block_height = 30

class Block:
    def __init__(self, x, y, letter, color=GREY):
        self.x = x
        self.y = y
        self.letter = letter.upper()
        self.width = block_width
        self.height = block_height
        self.color = color

    def draw(self, screen, camera_y):
        rect = pygame.Rect(self.x, self.y - camera_y, self.width, self.height)
        pygame.draw.rect(screen, self.color, rect)
        pygame.draw.rect(screen, (100, 100, 100), rect, 2)

        # ตัวอักษรตรงกลางบล็อก
        text = FONT.render(self.letter, True, BLACK)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)

blocks = []

def add_word_blocks(word):
    """เพิ่มบล็อกจากคำตอบที่ถูก ต่อเป็นแนวตั้ง"""
    if len(blocks) == 0:
        base_y = HEIGHT - 130
    else:
        top_block = min(block.y for block in blocks)
        base_y = top_block - block_height

    x_center = WIDTH // 2 - block_width // 2

    for i, ch in enumerate(word):
        blocks.append(Block(x_center, base_y - i * block_height, ch))

def draw_text(surface, text, font, color, x, y):
    txt = font.render(text, True, color)
    surface.blit(txt, (x, y))

def draw_wave(surface, water_level, camera_y, amplitude=10, wavelength=150, speed=2):
    """วาดเอฟเฟกต์น้ำกระเพื่อม"""
    points = []
    time_now = pygame.time.get_ticks() / 1000
    for x in range(0, WIDTH + 10, 10):
        y = (water_level - camera_y) + math.sin((x / wavelength) + time_now * speed) * amplitude
        points.append((x, y))
    if len(points) > 1:
        pygame.draw.lines(surface, LIGHT_BLUE, False, points, 3)

def main(): 
    running = True
    input_text = ""
    question = random.choice(list(questions.keys()))
    current_question = questions[question]

    # --- กล้อง ---
    camera_y = 0
    tower_height = 0
    camera_box_height = HEIGHT // 2  # ความสูงกล้องที่มองได้
    camera_target_y = 0

    # --- น้ำ ---
    water_level = HEIGHT - 100
    base_rise_speed = 0.1
    water_speed = base_rise_speed

    # --- คะแนน ---
    score = 0

    # --- Game Over ---
    game_over = False

    while running:
        screen.fill(BLACK)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_RETURN:
                        if input_text.lower() in current_question:
                            add_word_blocks(input_text)
                            tower_height = len(blocks) * block_height
                            score += len(input_text) * 10
                            input_text = ""
                            question = random.choice(list(questions.keys()))
                            current_question = questions[question]
                        else:
                            input_text = ""
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        input_text += event.unicode
                else:
                    if event.key == pygame.K_r:
                        blocks.clear()
                        input_text = ""
                        question = random.choice(list(questions.keys()))
                        current_question = questions[question]
                        water_level = HEIGHT - 100
                        water_speed = base_rise_speed
                        tower_height = 0
                        camera_y = 0
                        score = 0
                        game_over = False

        # --- อัปเดตกล้องแบบ Box ---
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            bottom_block_y = max(block.y for block in blocks)
            tower_top_in_view = top_block_y - camera_y
            tower_bottom_in_view = bottom_block_y - camera_y

            # ถ้า tower เลยกรอบบน/ล่าง ให้กล้องขยับ
            if tower_top_in_view < HEIGHT / 4:
                camera_target_y = top_block_y - HEIGHT / 4
            elif tower_bottom_in_view > HEIGHT * 3 / 4:
                camera_target_y = bottom_block_y - HEIGHT * 3 / 4
        else:
            camera_target_y = 0

        camera_y += (camera_target_y - camera_y) * 0.1  # smooth move

        # --- น้ำขึ้น ---
        water_speed = base_rise_speed + (tower_height / 5000)
        water_level -= water_speed

        # --- ตรวจสอบ Game Over ---
        if len(blocks) > 0:
            top_block_y = min(block.y for block in blocks)
            if water_level <= top_block_y:
                game_over = True

        # --- วาดบล็อก ---
        for block in blocks:
            block.draw(screen, camera_y)

        # --- วาดน้ำ ---
        water_rect = pygame.Surface((WIDTH, HEIGHT - (water_level - camera_y)), pygame.SRCALPHA)
        water_rect.fill(BLUE)
        screen.blit(water_rect, (0, water_level - camera_y))
        draw_wave(screen, water_level, camera_y)

        # --- กล่อง input ---
        input_box = pygame.Rect(20, 20, 760, 50)
        pygame.draw.rect(screen, WHITE, input_box, 2)
        draw_text(screen, f"Question: {question}", FONT, WHITE, 20, 80)
        draw_text(screen, input_text, FONT, YELLOW, input_box.x + 10, input_box.y + 10)

        # --- คะแนน ---
        draw_text(screen, f"Score: {score}", FONT, GREEN, WIDTH - 200, 20)

        # --- Game Over ---
        if game_over:
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            draw_text(screen, "GAME OVER!", BIG, RED, WIDTH // 2 - 180, HEIGHT // 3)
            draw_text(screen, f"Final Score: {score}", FONT, WHITE, WIDTH // 2 - 120, HEIGHT // 2)
            draw_text(screen, "Press R to Restart", FONT, LIGHT_BLUE, WIDTH // 2 - 150, HEIGHT // 2 + 50)

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

if __name__ == "__main__":
    main()

error: Invalid resolution for Surface

In [ ]:
import pygame import math # (ลบ import random ออก) # --- ค่าคงที่ของเกม --- WIDTH, HEIGHT = 800, 600 BLOCK_SIZE = 20 BACKGROUND_COLOR = (10, 20, 40) WATER_COLOR = (20, 80, 200, 200) SURFACE_HIGHLIGHT = (100, 150, 255, 100) WATER_ANIM_DURATION = 0.9 WATER_SURFACE_WAVE_AMPLITUDE = 6 WATER_SURFACE_LENGTH = 140 round_number = 1 PERCENT_START = 0.50 PERCENT_STEP = 0.05 PERCENT_MAX = 1.0 WATER_RISE_PIXELS_PER_LETTER = 10 # เพิ่มให้ครบ # --- คลาส typingText --- class typingText: def __init__(self, screen, font, question, valid_answers): self.screen = screen self.font = font self.question = question self.valid_answers = set(ans.lower() for ans in valid_answers) self.user_input = "" self.message = "" self.message_color = (255, 255, 255) def handle_event(self, event): submitted = False if event.type == pygame.KEYDOWN: if event.key == pygame.K_BACKSPACE: self.user_input = self.user_input[:-1] elif event.key == pygame.K_RETURN: if self.user_input: submitted = True else: char = event.unicode if char.isprintable(): self.user_input += char return submitted def check_answer(self): is_correct = self.user_input.lower() in self.valid_answers if is_correct: self.message = "Correct!" self.message_color = (0, 255, 0) else: self.message = "Invalid or Unknown Word!" self.message_color = (255, 0, 0) return is_correct def get_answer_length(self): return len(self.user_input) def clear_input(self): self.user_input = "" self.message = "" def draw(self): question_surf = self.font.render(f"Question: {self.question}", True, (255, 255, 255)) self.screen.blit(question_surf, (20, 20)) input_rect = pygame.Rect(20, 60, WIDTH - 40, 50) pygame.draw.rect(self.screen, (255, 255, 255), input_rect, 2) input_surf = self.font.render(self.user_input, True, (255, 255, 0)) self.screen.blit(input_surf, (input_rect.x + 10, input_rect.y + 10)) result_surf = self.font.render(self.message, True, self.message_color) self.screen.blit(result_surf, (20, 120)) # --- คลาส Water --- class Water: def __init__(self, screen_h): self.screen_h = screen_h self.level_y = float(screen_h) self.visible = False self.animating = False self.start_y = self.level_y self.target_y = self.level_y self.anim_time = 0.0 self.anim_dur = WATER_ANIM_DURATION self.word_lengths = [] def start_rise(self, rise_pixels): if not self.visible: self.visible = True self.start_y = float(self.level_y) self.target_y = max(0.0, self.start_y - rise_pixels) self.anim_time = 0.0 self.animating = True def update(self, dt): if self.animating: self.anim_time += dt t = min(1.0, self.anim_time / self.anim_dur) t_ease = t * t * (3 - 2 * t) self.level_y = self.start_y + (self.target_y - self.start_y) * t_ease if t >= 1.0: self.level_y = self.target_y self.animating = False def draw(self, surf, camera_y=0): if not self.visible: return level = int(self.level_y - camera_y) if level < self.screen_h: water_surface = pygame.Surface((WIDTH, self.screen_h - level), pygame.SRCALPHA) water_surface.fill(WATER_COLOR) surf.blit(water_surface, (0, level)) points = [] t = pygame.time.get_ticks() / 1000.0 amp = WATER_SURFACE_WAVE_AMPLITUDE length = WATER_SURFACE_LENGTH for x in range(0, WIDTH + 8, 8): y = level + math.sin((x / length) + t * 1.5) * amp points.append((x, y)) if points: pygame.draw.lines(surf, (255, 255, 255), False, points, 3) def is_animating(self): return self.animating def reset(self): self.__init__(self.screen_h) # --- คลาส Scoring --- class Scoring: def __init__(self): self.char_score = 0 self.word_score = 0 self.words_history = [] self.is_game_over = False def register_word(self, word: str): if self.is_game_over: return clean_word = word.strip() if not clean_word: return self.char_score += len(clean_word) self.word_score += 1 self.words_history.append(clean_word) def game_over(self): self.is_game_over = True return self.get_final_score() def get_final_score(self): total = self.char_score + self.word_score return { "char_score": self.char_score, "word_score": self.word_score, "total_score": total, "words_history": self.words_history, } def reset(self): self.char_score = 0 self.word_score = 0 self.words_history = [] self.is_game_over = False # --- คลาส Player --- class Player: def __init__(self, screen_w, screen_h, block_size): self.screen_w = screen_w self.screen_h = screen_h self.block_size = block_size self.width = 100 self.x_pos = (self.screen_w // 2) - (self.width // 2) self.color = (200, 200, 200) self.survivor_color = (255, 0, 0) self.total_height_pixels = self.block_size self.y_pos = self.screen_h - self.total_height_pixels def add_height(self, num_letters): self.total_height_pixels += num_letters * self.block_size self.y_pos = self.screen_h - self.total_height_pixels def get_top_y(self): return (self.screen_h - self.total_height_pixels) - 20 def draw(self, surf, camera_y=0): draw_height = min(self.total_height_pixels, self.screen_h * 3) draw_y = self.screen_h - draw_height - camera_y player_rect = pygame.Rect(self.x_pos, draw_y, self.width, draw_height) pygame.draw.rect(surf, self.color, player_rect) survivor_y = self.get_top_y() - camera_y survivor_rect = pygame.Rect(self.x_pos + (self.width // 2) - 10, survivor_y, 20, 20) pygame.draw.rect(surf, self.survivor_color, survivor_rect) def reset(self): self.__init__(self.screen_w, self.screen_h, self.block_size) # --- ฟังก์ชันหลัก --- def main(): pygame.init() screen = pygame.display.set_mode((WIDTH, HEIGHT)) pygame.display.set_caption("Text or Die - Camera Mode") main_font = pygame.font.Font(None, 40) score_font = pygame.font.Font(None, 50) small_font = pygame.font.Font(None, 24) clock = pygame.time.Clock() valid_answers = [ "apple", "banana", "orange", "grape", "mango", "melon", "kiwi", "pear", "peach", "plum", "cherry", "berry", "lime", "lemon" ] question = "Name a fruit" player = Player(WIDTH, HEIGHT, BLOCK_SIZE) water = Water(HEIGHT) scoring = Scoring() typer = typingText(screen, main_font, question, valid_answers) running = True game_over = False game_won = False final_score_data = {} # --- ตัวแปรกล้อง --- camera_y = 0.0 camera_target_y = 0.0 while running: dt = clock.tick(60) / 1000.0 for event in pygame.event.get(): if event.type == pygame.QUIT: running = False if game_over: if event.type == pygame.KEYDOWN and event.key == pygame.K_r: player.reset() water.reset() scoring.reset() typer.clear_input() game_over = False game_won = False final_score_data = {} camera_y = 0.0 camera_target_y = 0.0 else: if not water.is_animating(): submitted = typer.handle_event(event) if submitted: is_correct = typer.check_answer() letters = typer.get_answer_length() water_rise_letters = 0.0 if is_correct: scoring.register_word(typer.user_input) player.add_height(letters) water_rise_letters = letters / 2.0 if player.get_top_y() <= 0: game_over = True game_won = True final_score_data = scoring.game_over() typer.clear_input() continue water.word_lengths.append(letters) else: water.word_lengths.append(letters) percent = PERCENT_START + PERCENT_STEP * (round_number - 1) percent = min(percent, PERCENT_MAX) if len(water.word_lengths) > 0: avg_len = sum(water.word_lengths) / len(water.word_lengths) else: avg_len = 0.0 water_rise_letters = avg_len * percent if water_rise_letters > 0 and not game_over: rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER water.start_rise(rise_px) typer.clear_input() if not game_over: water.update(dt) if not water.is_animating() and water.visible and water.level_y <= player.get_top_y(): game_over = True game_won = False final_score_data = scoring.game_over() # --- อัปเดตตำแหน่งกล้อง --- player_top = player.get_top_y() # กล้องจะเลื่อนตามหัวผู้เล่น โดยมีระยะเว้นข้างบน 200px camera_target_y = max(0, -player_top + 200) camera_y += (camera_target_y - camera_y) * 5.0 * dt # smooth follow # --- วาด --- screen.fill(BACKGROUND_COLOR) player.draw(screen, camera_y) water.draw(screen, camera_y) typer.draw() if game_over: overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA) overlay.fill((0, 0, 0, 180)) screen.blit(overlay, (0, 0)) if game_won: go_text = score_font.render("YOU REACHED THE TOP!", True, (0, 255, 100)) else: go_text = score_font.render("GAME OVER", True, (255, 0, 0)) screen.blit(go_text, (WIDTH // 2 - go_text.get_width() // 2, HEIGHT // 3)) restart_text = small_font.render("Press 'R' to Restart", True, (150, 150, 150)) screen.blit(restart_text, (WIDTH // 2 - restart_text.get_width() // 2, HEIGHT - 50)) pygame.display.flip() pygame.quit() if __name__ == "__main__": main()